# CSCI 3220: Python for Data Science and AI

## Assignment 3

## Due Date: April 11(Monday), 11:59 PM

## Total Points: 35

## Bonus Points: 5

#### ***Write the code in Jupyter notebook (ipynb file) with proper comments. 
#### ***Add proper citation if you take help from a different source (not from the textbook).
#### ***Rename the file with your student ID and submit it in Moodle.

In [16]:
import pandas as pd
import numpy as np
import sklearn
import math
import collections

## 1a.	Load any one of the toy datasets (for classification) from scikit-learn library.

In [2]:
from sklearn.datasets import load_iris

iris = load_iris()

## 1b. Display general information and statistics about your selected dataset.

In [3]:
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

## 2.	Split the dataset into train and test sets.

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(iris.data, iris.target)

## 3.	Implement a k-nearest neighbors classifier (k =3) for the training set. Show the result for the test set using the following metric: accuracy, precision, and recall.

In [65]:
#Gets Distance Between Two Observations
def getDistance(point1, point2):    
    distance_sepal_length = ((point1[0] - point2[0]) ** 2)
    distance_sepal_width = ((point1[1] - point2[1]) ** 2)
    distance_petal_length = ((point1[2] - point2[2]) ** 2)
    distance_petal_width = ((point1[3] - point2[3]) ** 2)
    distance = math.sqrt(distance_sepal_length + distance_sepal_width + distance_petal_length + distance_petal_width) 
    return distance
    
def getClosestNSpecies(point, x_train, y_train, n):
    bob = dict()
    
    #Add Distance to Dictionary Observations
    for train_observation in range(len(x_train)):
        bob.update({getDistance(point, x_train[train_observation]) : train_observation})
        
    #Citation: Learned How to Sort Dictionary on StackOverflow
    #Link: https://stackoverflow.com/questions/9001509/how-can-i-sort-a-dictionary-by-key
    bob = collections.OrderedDict(sorted(bob.items()))
    bob = dict(bob)
    
    #Increments Index of Species Array to Lowest Three Observation's Label
    species = [0, 0, 0]
    for i in range(n):
        train_observation_index = list(bob.values())[i]
        species[y_train[train_observation_index]] = species[y_train[train_observation_index]] + 1
     
    #Determine Which Species is Nearest Neighbors
    current_largest_n_species = -1 
    for i in range(len(species)):  
        if(species[i] > current_largest_n_species):
            current_largest_n_species = i
    
    return current_largest_n_species



getClosestNSpecies(x_test[0], x_train, y_train, 3)

1

## 4.	Split the training set into 10-fold for cross-validation. Calculate accuracy for each held-out set. Also, display the average accuracy for ten folds.

In [22]:
print(y_train)

[0 2 0 0 2 2 1 1 0 0 0 1 2 2 1 0 0 0 2 1 0 2 0 0 1 1 2 0 2 2 2 0 0 1 1 0 0
 1 1 2 0 1 0 1 1 0 1 1 0 2 1 1 2 2 0 2 1 2 0 0 2 2 2 2 2 2 2 0 0 2 0 2 1 0
 2 2 0 2 1 0 1 0 1 1 1 2 2 2 2 1 1 1 1 0 1 0 1 0 0 1 1 2 1 0 2 2 0 1 2 0 2
 0]


## 5.	Find the optimal value of k ranging from 1 to 10 based on mean accuracy of 10-fold cross-validation.

## 6. Implement the steps (3-5) with the scikit-learn library method and compare it with your previous results.